In [ ]:
!nvidia-smi


Thu Mar 20 22:55:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import cv2
import torch
import numpy as np
import time
from google.colab import drive

In [ ]:
model_path = "/content/yolov8s.ptl"  # Change to your model path
model = torch.jit.load(model_path)
model.eval()  #

RecursiveScriptModule(
  original_name=DetectionModel
  (model): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (1): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (2): RecursiveScriptModule(
      original_name=C2f
      (cv1): RecursiveScriptModule(
        original_name=Conv
        (conv): RecursiveScriptModule(original_name=Conv2d)
        (act): RecursiveScriptModule(original_name=SiLU)
      )
      (cv2): RecursiveScriptModule(
        original_name=Conv
        (conv): RecursiveScriptModule(original_name=Conv2d)
        (act): RecursiveScriptModule(original_name=SiLU)
      )
      (m): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModu

In [ ]:
video_input_path = "/content/TEST.mp4"  # Change to your video path
video_output_path = "/content/drive/My Drive/output_video.mp4"

In [ ]:
cap = cv2.VideoCapture(video_input_path)

# **Get video properties**
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

# **Variables to store previous drone position**
prev_center = None
prev_time = time.time()

# **Fix detect_objects function**
def detect_objects(image):
    """Runs inference on the image and returns bounding boxes."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image_resized = cv2.resize(image_rgb, (640, 640))  # Ensure consistent input size
    image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() / 255.0  # Convert to tensor
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension

    # Ensure tensor is on the correct device (CPU or GPU)
    device = next(model.parameters()).device
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        results = model(image_tensor)  # Run inference

    # Extract bounding boxes from results
    if isinstance(results, tuple) or isinstance(results, list):
        results = results[0]  # Some models return a tuple/list, get first item

    if results.dim() == 3:  # Ensure correct dimension handling
        results = results.squeeze(0)

    boxes = results[:, :4]  # Extract bounding box coordinates (x1, y1, x2, y2)
    return boxes.cpu().numpy()

# **STEP 5: Process Video Frame by Frame**
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection using the PTL model
    try:
        boxes = detect_objects(frame)
    except Exception as e:
        print(f"Error in detection: {e}")
        continue  # Skip frame if there's an error

    current_center = None
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)

        # Compute the center of the bounding box
        current_center = ((x1 + x2) // 2, (y1 + y2) // 2)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # **Compute Speed Classification**
    speed_category = "Static"  # Default
    if prev_center is not None and current_center is not None:
        curr_time = time.time()
        delta_t = curr_time - prev_time  # Time difference

        if delta_t > 0:  # Ensure no division by zero
            displacement = np.linalg.norm(np.array(current_center) - np.array(prev_center))  # Euclidean distance

            # **Define Speed Thresholds (Adjust as needed)**
            if displacement < 5:
                speed_category = "Static"
            elif displacement < 20:
                speed_category = "Slow"
            elif displacement < 50:
                speed_category = "Medium"
            else:
                speed_category = "Fast"

        prev_time = curr_time  # Update time

    # Update previous position
    prev_center = current_center

    # Display speed category
    cv2.putText(frame, f"Speed: {speed_category}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # **Write Frame to Output Video**
    out.write(frame)

# **STEP 6: Save and Release Resources**
cap.release()
out.release()

print(f"✅ Video processing complete! Saved at: {video_output_path}")


✅ Video processing complete! Saved at: /content/drive/My Drive/output_video.mp4
